In [1]:
from Cryptodome.Cipher import AES
from Cryptodome.Hash import SHA256
from Cryptodome.Util.Padding import unpad
from Cryptodome.Util.Padding import pad
from cypari import pari

In [5]:
from random import randrange
from math import gcd
import time
import base64
import sys
bs = AES.block_size
high = 2 ** 512
low = 2 ** 511
_seq = 0x30.to_bytes(1, byteorder='big')
_set = 0x31.to_bytes(1, byteorder='big')
_oct = 0x04.to_bytes(1, byteorder='big')
_int = 0x02.to_bytes(1, byteorder='big')

def byte_length(a):
    b = a.bit_length()
    if b % 8 != 0:
        return (b // 8) + 1
    else:
        return b // 8
def len_to_bytes(a):
    if a < 128:
        return a.to_bytes(1, byteorder='big')
    else:
        b = (128 + byte_length(a)).to_bytes(1, byteorder='big')
        b = b + a.to_bytes(byte_length(a), byteorder='big')
        return b
def pack_int(a):
    pack = _int
    if a // (2 ** (byte_length(a) * 8 - 1)) == 1:
        pack = pack + len_to_bytes(byte_length(a) + 1) + (0x00).to_bytes(1, byteorder='big')
    else:
        pack = pack + len_to_bytes(byte_length(a))
    pack = pack + a.to_bytes(byte_length(a), byteorder='big')
    return pack
def bytes_to_int(a):
    if type(a) == int:
        return a
    else:
        result = 0
        for i in range(0, len(a)):
            result = result * 256 + a[i]
        return result
def bytes_to_len(a):
    if a[0] >> 7:
        return bytes_to_int(a[1:1 + (a[0] & 0x7F)])
    else:
        return a[0]
def len_of_len(a):
    if a[0] >> 7:
        return 1 + (a[0] & 0x7F)
    else:
        return 1
def gen_param():
    p_gen = pari(randrange(low, high))
    while not p_gen.isprime():
        if p_gen.nextprime(1) > high:
            p_gen = low.nextprime(1)
        else:
            p_gen = p_gen.nextprime(1)
    p = int(p_gen)
    q_gen = pari(randrange(low, high))
    while not q_gen.isprime():
        if q_gen.nextprime(1) > high:
            q_gen = low.nextprime(1)
        else:
            q_gen = q_gen.nextprime(1)
    q = int(q_gen)
    n = p * q
    euler = (p - 1) * (q - 1)
    e = randrange(2, euler - 1)
    while gcd(euler, e) != 1:
        if e == euler - 2:
            e = 2
        else:
            e += 1
    d = int((pari(1)/pari(e)).Mod(pari(euler)).lift())
    return p,q,n,euler,e,d
    
def to_asn(key, n, e,d, c, iv, m, flag):
    if flag==1:
        enc_key = int(pari(key).Mod(n)**pari(e))
    else:
        enc_key = int(pari(key).Mod(n)**pari(d))
    rsaid_pack = _oct + len_to_bytes(2) + (0x0001).to_bytes(2, byteorder='big')
    n_pack = pack_int(n)
    e_pack = pack_int(e)
    public_pack = _seq + len_to_bytes(len(n_pack) + len(e_pack)) + n_pack + e_pack
    param_pack = _seq + len_to_bytes(0)
    key_pack = pack_int(enc_key)
    cypher_pack = _seq + len_to_bytes(len(key_pack)) + key_pack
    first_pack = _seq + len_to_bytes(len(rsaid_pack) + len(public_pack) + len(param_pack) + len(cypher_pack)) + rsaid_pack + public_pack + param_pack + cypher_pack
    keys_pack = _set + len_to_bytes(len(first_pack)) + first_pack
    if flag==1:
        aesid_pack = _oct + len_to_bytes(2) + (0x1081).to_bytes(2, byteorder='big')
        len_pack = pack_int(len(m))
        iv_pack = pack_int(iv)
        aes_pack = _seq + len_to_bytes(len(aesid_pack) + len(len_pack) + len(iv_pack)) + aesid_pack + len_pack + iv_pack
        header_pack = _seq + len_to_bytes(len(keys_pack) + len(aes_pack)) + keys_pack + aes_pack
        return header_pack + c
    else:
        add_pack = _seq + len_to_bytes(0)
        return _seq + len_to_bytes(len(keys_pack) + len(add_pack)) + keys_pack + add_pack
    
def encrypt(m, output_filename, private_filename):
    start_time = time.time()
    p,q,n,euler,e,d=gen_param()
    stop_time = time.time() - start_time
    print("Keys generated in ", stop_time, " seconds.")
    print("p=", p)
    print("q=", q)
    print("n=", n)
    print("e=", e)
    print("d=", d)
    start_time = time.time()
    key = randrange(0, 2**256)
    iv = randrange(0, 2**128)
    m = pad(m, bs)
    aes = AES.new(key.to_bytes(32, byteorder='big'), AES.MODE_CBC, iv.to_bytes(16, byteorder='big'))
    c = base64.b64encode(iv.to_bytes(16, byteorder='big') + aes.encrypt(m))  
    asn=to_asn(key, n, e, d, c, iv, m,1)
    stop_time = time.time() - start_time
    print("Data encrypted in ", stop_time, " seconds.")
    output_file = open(output_filename, "wb")
    output_file.write(asn)
    output_file.close()
    private_file = open(private_filename, "wb")
    private_file.write(d.to_bytes(byte_length(d), byteorder='big'))
    private_file.close()

def from_asn(asn, flag):
    if flag==1:
        header_len = bytes_to_len(asn[1:]) + len_of_len(asn[1:]) + 1
        header_pack = asn[:header_len]
        header_pack = header_pack[len_of_len(header_pack[1:]) + 1:]
        c = asn[header_len:]
        keys_len = bytes_to_len(header_pack[1:]) + len_of_len(header_pack[1:]) + 1
        keys_pack = header_pack[:keys_len]
        keys_pack = keys_pack[len_of_len(keys_pack[1:]) + 1:]
        aes_pack = header_pack[keys_len:]
        aes_pack = aes_pack[len_of_len(aes_pack[1:]) + 1:]
        first_pack = keys_pack[len_of_len(keys_pack[1:]) + 1:]
        rsaid_len = bytes_to_len(first_pack[1:]) + len_of_len(first_pack[1:]) + 1
        first_pack = first_pack[rsaid_len:]
        public_len = bytes_to_len(first_pack[1:]) + len_of_len(first_pack[1:]) + 1
        public_pack = first_pack[:public_len]
        public_pack = public_pack[len_of_len(public_pack[1:]) + 1:]
        first_pack = first_pack[public_len:]
        param_len = bytes_to_len(first_pack[1:]) + len_of_len(first_pack[1:]) + 1
        cypher_pack = first_pack[param_len:]
        cypher_pack = cypher_pack[len_of_len(cypher_pack[1:]) + 1:]
        key_pack = cypher_pack[len_of_len(cypher_pack[1:]) + 1:]
        n_len = bytes_to_len(public_pack[1:]) + len_of_len(public_pack[1:]) + 1
        n_pack = public_pack[:n_len]
        n_pack = n_pack[len_of_len(n_pack[1:]) + 1:]
        e_pack = public_pack[n_len:]
        e_pack = e_pack[len_of_len(e_pack[1:]) + 1:]
        aesid_len = bytes_to_len(aes_pack[1:]) + len_of_len(aes_pack[1:]) + 1
        aes_pack = aes_pack[aesid_len:]
        len_len = bytes_to_len(aes_pack[1:]) + len_of_len(aes_pack[1:]) + 1
        len_pack = aes_pack[:len_len]
        len_pack = len_pack[len_of_len(len_pack[1:]) + 1:]
        iv_pack = aes_pack[len_len:]
        iv_pack = iv_pack[len_of_len(iv_pack[1:]) + 1:]
        enc_key = bytes_to_int(key_pack)
        iv = bytes_to_int(iv_pack)
        d = bytes_to_int(private)
        n = bytes_to_int(n_pack)
        key = int(pari(enc_key).Mod(n)**pari(d))
        return key,n,d,iv,c
    else:
        header_len = bytes_to_len(asn[1:]) + len_of_len(asn[1:]) + 1
        header_pack = asn[:header_len]
        header_pack = header_pack[len_of_len(header_pack[1:]) + 1:]
        keys_len = bytes_to_len(header_pack[1:]) + len_of_len(header_pack[1:]) + 1
        keys_pack = header_pack[:keys_len]
        keys_pack = keys_pack[len_of_len(keys_pack[1:]) + 1:]
        add_pack = header_pack[keys_len:]
        add_pack = add_pack[len_of_len(add_pack[1:]) + 1:]
        first_pack = keys_pack[len_of_len(keys_pack[1:]) + 1:]
        rsaid_len = bytes_to_len(first_pack[1:]) + len_of_len(first_pack[1:]) + 1
        first_pack = first_pack[rsaid_len:]
        public_len = bytes_to_len(first_pack[1:]) + len_of_len(first_pack[1:]) + 1
        public_pack = first_pack[:public_len]
        public_pack = public_pack[len_of_len(public_pack[1:]) + 1:]
        first_pack = first_pack[public_len:]
        param_len = bytes_to_len(first_pack[1:]) + len_of_len(first_pack[1:]) + 1
        cypher_pack = first_pack[param_len:]
        cypher_pack = cypher_pack[len_of_len(cypher_pack[1:]) + 1:]
        sig_pack = cypher_pack[len_of_len(cypher_pack[1:]) + 1:]
        n_len = bytes_to_len(public_pack[1:]) + len_of_len(public_pack[1:]) + 1
        n_pack = public_pack[:n_len]
        n_pack = n_pack[len_of_len(n_pack[1:]) + 1:]
        e_pack = public_pack[n_len:]
        e_pack = e_pack[len_of_len(e_pack[1:]) + 1:]
        signature = bytes_to_int(sig_pack)
        e = bytes_to_int(e_pack)
        n = bytes_to_int(n_pack)
        return signature, m,n,e
    
def decrypt(asn, private, output_filename):
    start_time = time.time()
    key,n,d,iv,c=from_asn(asn,1)    
    aes = AES.new(key.to_bytes(32, byteorder='big'), AES.MODE_CBC, iv.to_bytes(16, byteorder='big'))
    c = base64.b64decode(c)
    m = aes.decrypt(c[bs:])
    m = unpad(m, bs)
    stop_time = time.time() - start_time
    print("Data decrypted in ", stop_time, " seconds.")
    output_file = open(output_filename, "wb")
    output_file.write(m)
    output_file.close()
    
def sign(m, output_filename):
    start_time = time.time()    
    p,q,n,euler,e,d=gen_param()
    stop_time = time.time() - start_time
    print("Keys generated in ", stop_time, " seconds.")
    start_time = time.time()
    h = SHA256.new()
    h.update(m)
    #print (h)
    header_pack=to_asn(bytes_to_int(h.digest()), n, e,d, h, 0, m,2)
    stop_time = time.time() - start_time
    print("Data signed in ", stop_time, " seconds.")
    output_file = open(output_filename, "wb")
    #print('asn')
    #print (header_pack)
    output_file.write(header_pack)
    output_file.close()

def verify(m, asn):
    start_time = time.time()
    #print('asn')
    #print (asn)
    signature, m,n,e=from_asn(asn, 2)
    h = SHA256.new()
    h.update(m)
    h1 = int(pari(signature).Mod(n)**pari(e))
    #print (h1)
    #print (h)
    #print (bytes_to_int(h.digest()))
    if h1 == bytes_to_int(h.digest()):
        print("Verification successful.")
    else:
        print("Verification failed.")
    stop_time = time.time() - start_time
    print("Data decrypted in ", stop_time, " seconds.")


In [8]:
i=input('1-encrypt\n2-decrypt\n3-sign\n4-verify\n0-exit\n')
while i != '0':
    if i == '1':
        input_filename = input("File name:")
        input_file = open(input_filename, "rb")
        asn = input_file.read()
        input_file.close()
        encrypt(asn, "enc_" + input_filename, "private.txt")
    elif i == '2':
        input_filename = input("File name:")
        private_filename = input("private_filename:")
        input_file = open(input_filename, "rb")
        asn = input_file.read()
        input_file.close()
        private_file = open(private_filename, "rb")
        private = private_file.read()
        private_file.close()
        decrypt(asn, private, "dec_" + input_filename)
    elif i == '3':
        input_filename = input("File name:")
        input_file = open(input_filename, "rb")
        asn = input_file.read()
        input_file.close()
        sign(asn, "sig_" + input_filename,)
    elif i == '4':
        input_filename = input("File name:")
        signature_filename = input("signature_filename:")
        input_file = open(input_filename, "rb")
        m = input_file.read()
        input_file.close()
        signature_file = open(signature_filename, "rb")
        asn = signature_file.read()
        signature_file.close()
        verify(m, asn)
    i=input('1-encrypt\n2-decrypt\n3-sign\n4-verify\n0-exit\n')
exit()

1-encrypt
2-decrypt
3-sign
4-verify
0-exit
4
File name:text.txt
signature_filename:sig_text.txt
57488675851491450905663997722243775579995850568419986001065510490483088498266
Verification failed.
Data decrypted in  0.0010013580322265625  seconds.
1-encrypt
2-decrypt
3-sign
4-verify
0-exit
1
File name:text.txt
Keys generated in  0.4721064567565918  seconds.
p= 13353558698350854471970220322215919772971970879462472489698162734519014745110145294712105355747946596999526703917728561012251668122180759403150813260599553
q= 8240437892624872432228758247211632295348247193601241089132269836576562056774888872843666663759579647447120366517309774916069133446498287308020640808571529
n= 110039171099280849804149044943827009645508713332215093043206404183651973173688379554555452635337816955621986101803859018418604258216488467526400916493412612220867761254354223995073291626864998032460283188005641813155906064822653078772882338629225689885638750086414057387172833808920969502852594490376025926537
e= 592905552

KeyboardInterrupt: Interrupted by user